# Var Config

In [5]:
import pandas as pd

In [6]:
fb_data = pd.read_csv(r"C:\Users\Owner\Documents\Data Projects\GitHub\Apps\project_w\data\football_data.csv")

In [7]:
team = 'Aston Villa'

In [8]:
team_url = fb_data[fb_data['team'] == team].url.values[0]
team_url

'https://ng.soccerway.com//teams/england/aston-villa-football-club/665/'

## Requests

#### (currently (10/7/24) not working (but is currentl what we're using in the class because it worked before))

In [12]:
import requests
from bs4 import BeautifulSoup

In [15]:
agent = 'Mozilla/5.0 (Windows NT 10.0; Windows; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.114 Safari/537.36'

In [16]:
team_response = requests.get(team_url, headers={'User-Agent': agent})

In [17]:
team_response

<Response [403]>

## Selenium + BS4

In [9]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup

In [10]:
# Get the ChromeDriver path from your environment variable
chrome_driver_path = os.getenv('chrome_driver_path')

# Setup WebDriver
service = Service(chrome_driver_path)  # Use the path from environment variable
driver = webdriver.Chrome(service=service)

### Test out Selenium

In [27]:
try:
    # Open the page
    driver.get(team_url)
    
    # Wait until the cookie popup is present and clickable
    WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//button[text()='Reject All']"))
    ).click()

    # Now proceed with your scraping task
    print(driver.title)

except TimeoutException:
    print("Timed out waiting for cookie pop-up or other elements")
finally:
    # Close the browser
    driver.quit()

England - Aston Villa FC - Results, fixtures, squad, statistics, photos, videos and news - Soccerway


### Test out Selenium + BS4

In [11]:
try:
    # Open the page
    driver.get(team_url)
    
    # Wait until the cookie popup is present and clickable
    WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//button[text()='Reject All']"))
    ).click()

    # Now proceed with your scraping task
    ## Use BeautifulSoup to parse the page source once the page is fully loaded
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Find the table and extract data as before
    table = soup.find_all('table', class_='matches')
    dates = [row.text for row in table[0].find_all('td', class_="full-date")]
    leagues = [row.text.strip() for row in table[0].find_all('td', class_="competition")]
    homes = [row.text.strip() for row in table[0].find_all('td', class_="team")[::2]]
    aways = [row.text.strip() for row in table[0].find_all('td', class_="team")[1::2]]
    times = [row.text.strip() for row in table[0].find_all('td', class_="score-time")]

    # Create dataframes
    df_fixtures = pd.DataFrame(
        {
            'Date': dates,
            'League': leagues,
            'Home team': homes,
            'Time': times,
            'Away team': aways
        }
    )

except TimeoutException:
    print("Timed out waiting for cookie pop-up or other elements")
finally:
    # Close the browser
    driver.quit()

In [12]:
df_fixtures

,Date,League,Home team,Time,Away team
0,21/09/24,PRL,Aston Villa,3 - 1,Wolverhampton Wanderers
1,24/09/24,LEC,Wycombe Wanderers,1 - 2,Aston Villa
2,29/09/24,PRL,Ipswich Town,2 - 2,Aston Villa
3,02/10/24,UCL,Aston Villa,1 - 0,Bayern Munich
4,06/10/24,PRL,Aston Villa,0 - 0,Manchester United
5,19/10/24,PRL,Fulham,08 : 00,Aston Villa
6,22/10/24,UCL,Aston Villa,13 : 00,Bologna
7,26/10/24,PRL,Aston Villa,08 : 00,AFC Bournemouth
8,30/10/24,LEC,Aston Villa,13 : 45,Crystal Palace
9,03/11/24,PRL,Tottenham Hotspur,07 : 00,Aston Villa
